In [4]:
import random

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

import transformers
from transformers import AdamW, BertConfig, BertForSequenceClassification

from datasets import load_dataset, load_metric

# 加载数据

In [ ]:
task = 'cola'
mod